# Day 6: Wait For It

The ferry quickly brings you across Island Island. After asking around, you discover that there is indeed normally a large pile of sand somewhere near here, but you don't see anything besides lots of water and the small island where the ferry has docked.

As you try to figure out what to do next, you notice a poster on a wall near the ferry dock. "Boat races! Open to the public! Grand prize is an all-expenses-paid trip to _Desert Island_!" That must be where the sand comes from! Best of all, the boat races are starting in just a few minutes.

You manage to sign up as a competitor in the boat races just in time. The organizer explains that it's not really a traditional race - instead, you will get a fixed amount of time during which your boat has to travel as far as it can, and you win if your boat goes the farthest.

As part of signing up, you get a sheet of paper (your puzzle input) that lists the _time_ allowed for each race and also the best _distance_ ever recorded in that race. To guarantee you win the grand prize, you need to make sure you _go farther in each race_ than the current record holder.

The organizer brings you over to the area where the boat races are held. The boats are much smaller than you expected - they're actually _toy boats_, each with a big button on top. Holding down the button _charges the boat_, and releasing the button _allows the boat to move_. Boats move faster if their button was held longer, but time spent holding the button counts against the total race time. You can only hold the button at the start of the race, and boats don't move until the button is released.

For example:

```
Time:      7  15   30
Distance:  9  40  200
```

This document describes three races:

- The first race lasts 7 milliseconds. The record distance in this race is 9 millimeters.
- The second race lasts 15 milliseconds. The record distance in this race is 40 millimeters.
- The third race lasts 30 milliseconds. The record distance in this race is 200 millimeters.

Your toy boat has a starting speed of _zero millimeters per millisecond_. For each whole millisecond you spend at the beginning of the race holding down the button, the boat's speed increases by _one millimeter per millisecond_.

So, because the first race lasts 7 milliseconds, you only have a few options:

- Don't hold the button at all (that is, hold it for _`0` milliseconds_) at the start of the race. The boat won't move; it will have traveled _`0` millimeters_ by the end of the race.
- Hold the button for _`1` millisecond_ at the start of the race. Then, the boat will travel at a speed of `1` millimeter per millisecond for 6 milliseconds, reaching a total distance traveled of _`6` millimeters_.
- Hold the button for _`2` milliseconds_, giving the boat a speed of `2` millimeters per millisecond. It will then get 5 milliseconds to move, reaching a total distance of _`10` millimeters_.
- Hold the button for _`3` milliseconds_. After its remaining 4 milliseconds of travel time, the boat will have gone _`12` millimeters_.
- Hold the button for _`4` milliseconds_. After its remaining 3 milliseconds of travel time, the boat will have gone _`12` millimeters_.
- Hold the button for _`5` milliseconds_, causing the boat to travel a total of _`10` millimeters_.
- Hold the button for _`6` milliseconds_, causing the boat to travel a total of _`6` millimeters_.
- Hold the button for _`7` milliseconds_. That's the entire duration of the race. You never let go of the button. The boat can't move until you let go of the button. Please make sure you let go of the button so the boat gets to move. _`0` millimeters_.

Since the current record for this race is `9` millimeters, there are actually `_4_` different ways you could win: you could hold the button for `2`, `3`, `4`, or `5` milliseconds at the start of the race.

In the second race, you could hold the button for at least `4` milliseconds and at most `11` milliseconds and beat the record, a total of `_8_` different ways to win.

In the third race, you could hold the button for at least `11` milliseconds and no more than `19` milliseconds and still beat the record, a total of `_9_` ways you could win.

To see how much margin of error you have, determine the _number of ways you can beat the record_ in each race; in this example, if you multiply these values together, you get `_288_` (`4` \* `8` \* `9`).

Determine the number of ways you could beat the record in each race. _What do you get if you multiply these numbers together?_

In [60]:
$sample_data = @'
Time:      7  15   30
Distance:  9  40  200
'@ -split "`n" #NB: Polyglot seems to be OK with this, but in plain PoSH, it caused me pain as it had CRLF ("`r`n") line endings. FYI.

$input_data = @'
Time:        35     93     73     66
Distance:   212   2060   1201   1044
'@ -split "`n"

# Code
Hah, I think there's some cheaters; seems difficult to have been able to reach 9mm in the first race, haha...

In [118]:
function dostuff {
    [CmdletBinding()] Param(
        [Parameter(Mandatory=$false)]
        [switch] $sample,
        [Parameter(Mandatory=$false)]
        [switch] $part2
    )

    if($sample) {
        $data = $sample_data
    } else {
        $data = $input_data
    }

    $race_durations = [int[]][regex]::Matches($data[0], '\d+').value
    $race_highscore = [int[]][regex]::Matches($data[1], '\d+').value

    $best_ways_to_win = ,0 * $race_durations.count

    if($part2) {
        $race_durations = ,[bigint]([string[]]$race_durations -join '')
        $race_highscore = ,[bigint]([string[]]$race_highscore -join '')
        $best_ways_to_win = @(0)
        Write-Debug "Race duration: $race_durations, race highscore: $race_highscore"
    }


    for($x=0; $x -lt $race_durations.Count; $x++) {
        $duration = $race_durations[$x]
        $score = $race_highscore[$x]

        # The +/- 0.00000000001 is just a little hack to ensure if the math works out to be a whole number (e.g. an int), it'll still give us the next lower/higher number.
        $shortest_match = [Math]::Ceiling([double]((($duration - [math]::Sqrt([bigint]::Pow($duration, 2) - (4 * $score)))/2) + 0.00000000001)) 
        $longest_match = [Math]::Floor([double]((($duration + [math]::Sqrt([bigint]::Pow($duration, 2) - (4 * $score)))/2) - 0.00000000001))

        $best_ways_to_win[$x] = $longest_match - $shortest_match + 1

        Write-Debug "For race $($x), the shortest duration to beat $score is $shortest_match; while longest is $longest_match; i.e. $($best_ways_to_win[$x])"

    }

    $total = 1
    foreach($number in $best_ways_to_win) {
        $total *= $number
    }

    return $total
}

In [112]:
Write-Output "The product of all the better ways is $(dostuff -sample -Debug)"

DEBUG: For race 0, the shortest duration to beat 9 is 2; while longest is 5; i.e. 4
DEBUG: For race 1, the shortest duration to beat 40 is 4; while longest is 11; i.e. 8
DEBUG: For race 2, the shortest duration to beat 200 is 11; while longest is 19; i.e. 9
The product of all the better ways is 288


In [120]:
Write-Output "The product of all the better ways is $(dostuff -Debug)"

DEBUG: For race 0, the shortest duration to beat 212 is 8; while longest is 27; i.e. 20
DEBUG: For race 1, the shortest duration to beat 2060 is 37; while longest is 56; i.e. 20
DEBUG: For race 2, the shortest duration to beat 1201 is 26; while longest is 47; i.e. 22
DEBUG: For race 3, the shortest duration to beat 1044 is 27; while longest is 39; i.e. 13
The product of all the better ways is 114400


In [119]:
Write-Output "The product of all the better ways is $(dostuff -sample -Debug -part2)"
#Note: I can't completely explain why the solution here doesn't need the +1??

DEBUG: Race duration: 71530, race highscore: 940200
DEBUG: For race 0, the shortest duration to beat 940200 is 13; while longest is 71516; i.e. 71504
The product of all the better ways is 71504


In [121]:
Write-Output "The product of all the better ways is $(dostuff -Debug -part2)"
#Note: I can't completely explain why the solution here doesn't need the +1??

DEBUG: Race duration: 35937366, race highscore: 212206012011044
DEBUG: For race 0, the shortest duration to beat 212206012011044 is 7448818; while longest is 28488547; i.e. 21039730
The product of all the better ways is 21039730
